In [1]:
using LinearAlgebra
using Random
Random.seed!(4648373)
using TensorOperations

In [2]:
A = [
    1 3
    3 4
]

B = [
    5 6
    7 8
]

c = @tensor A[i, j] * B[j, i]

76

In [3]:
tr(A*B)

76

In [4]:
i = rand(5:13, 4)
j = rand(3:6, 4)
i[1] = 100
i, j

([100, 7, 8, 11], [4, 6, 5, 6])

In [5]:
G = [randn(i[mod1(k, 4)], j[k], i[mod1(k+1, 4)]) for k in 1:4]
size.(G)

4-element Vector{Tuple{Int64, Int64, Int64}}:
 (100, 4, 7)
 (7, 6, 8)
 (8, 5, 11)
 (11, 6, 100)

In [6]:
@macroexpand @tensor A[j1, j2, j3, j4] := G[1][i1, j1, i2] * G[2][i2, j2, i3] * G[3][i3, j3, i4] * G[4][i4, j4, i1]

quote
    var"##453" = G[1]
    var"##454" = G[2]
    var"##455" = G[3]
    var"##456" = G[4]
    var"##461" = promote_type(promote_type(promote_type(eltype(var"##453"), eltype(var"##454")), eltype(var"##455")), eltype(var"##456"), typeof(true))
    var"##465" = promote_type(promote_type(eltype(var"##453"), eltype(var"##454")), eltype(var"##455"), typeof(true))
    var"##469" = promote_type(eltype(var"##453"), eltype(var"##454"), typeof(true))
    var"##466" = var"##453"
    var"##467" = var"##454"
    var"##468" = TensorOperations.cached_similar_from_indices(Symbol("##468"), var"##469", (1, 2), (2, 3), (1, 2, 3), (4,), var"##466", var"##467", :N, :N)
    var"##462" = TensorOperations.contract!(true * true * true, var"##466", :N, var"##467", :N, false, var"##468", (1, 2), (3,), (2, 3), (1,), (1, 2, 3), (4,), (Symbol("##470"), Symbol("##471"), Symbol("##472")))
    var"##463" = var"##455"
    var"##464" = TensorOperations.cached_similar_from_indices(Symbol("##464"), var"##465", (1, 2, 3

In [7]:
@tensor A[j1, j2, j3, j4] := G[1][i1, j1, i2] * G[2][i2, j2, i3] * G[3][i3, j3, i4] * G[4][i4, j4, i1]
A

4×6×5×6 Array{Float64, 4}:
[:, :, 1, 1] =
 -98.1599    59.8415   -369.02   160.193    247.483  -246.628
  78.5465  -433.429     402.16   -40.5132  -412.02     85.0547
  10.6543    -9.78487  -157.838  -38.9479  -158.866   -43.12
 -72.2632   154.27      264.418  -91.4306  -132.694   -67.0245

[:, :, 2, 1] =
  28.3777    49.8404   -711.155   -703.158   -111.822    -45.7277
 144.26    -138.161    -328.717     58.6918   265.785    177.742
  57.9223     7.93006  -133.585   -330.248   -134.977    -71.5173
  97.5816  -104.529      60.7184   -24.9773    36.9373  -205.504

[:, :, 3, 1] =
  168.227    -32.8168   130.914   -59.2854   -30.9975   266.372
  176.259    217.786    -82.2031  -58.5456   185.44     248.358
  -24.8016  -121.249    360.955    25.5047  -172.363   -511.672
 -109.954    -13.3711  -238.422   177.019    -57.2132  -495.766

[:, :, 4, 1] =
 341.937    -384.331   145.145   -17.6244   -42.3556   163.255
  65.8122    128.402    36.3643  313.596     36.7675  -278.794
 102.005     345.

In [8]:
function refA_Gs(A, G, N)
    Gs = :($G[1][i1, j1, i2])
    for k in 2:N
        ik = Symbol(:i, k)
        jk = Symbol(:j, k)
        ikp1 = Symbol(:i, mod1(k+1, N))
        Gs = :($Gs * $G[$k][$ik, $jk, $ikp1])
    end
    refA = Expr(:ref, :A)
    for k in 1:N
        push!(refA.args, Symbol(:j, k))
    end
    refA, Gs
end

refA_Gs (generic function with 1 method)

In [9]:
refA_Gs(:A, :G, 4)

(:(A[j1, j2, j3, j4]), :((((G[1])[i1, j1, i2] * (G[2])[i2, j2, i3]) * (G[3])[i3, j3, i4]) * (G[4])[i4, j4, i1]))

In [10]:
macro multitrace(G, N)
    refA, Gs = refA_Gs(:A, :G, N)
    quote
        let G = $(esc(G))
            @tensor $refA := $Gs
        end
    end
end

@multitrace (macro with 1 method)

In [11]:
@macroexpand @multitrace G 4

quote
    #= In[10]:4 =#
    let var"#292#G" = G
        #= In[10]:5 =#
        begin
            var"#293###1129" = var"#292#G"[1]
            var"#294###1130" = var"#292#G"[2]
            var"#295###1131" = var"#292#G"[3]
            var"#296###1132" = var"#292#G"[4]
            var"#297###1137" = Main.promote_type(Main.promote_type(Main.promote_type(Main.eltype(var"#293###1129"), Main.eltype(var"#294###1130")), Main.eltype(var"#295###1131")), Main.eltype(var"#296###1132"), Main.typeof(true))
            var"#298###1141" = Main.promote_type(Main.promote_type(Main.eltype(var"#293###1129"), Main.eltype(var"#294###1130")), Main.eltype(var"#295###1131"), Main.typeof(true))
            var"#299###1145" = Main.promote_type(Main.eltype(var"#293###1129"), Main.eltype(var"#294###1130"), Main.typeof(true))
            var"#300###1142" = var"#293###1129"
            var"#301###1143" = var"#294###1130"
            var"#302###1144" = TensorOperations.cached_similar_from_indices(Symbol("##1144"), 

In [12]:
B = @multitrace G 4
A == B

true

In [13]:
macro multitrace(A, G, N)
    refA, Gs = refA_Gs(:A, :G, N)
    quote
        let A = $(esc(A)), G = $(esc(G))
            @tensor $refA = $Gs
        end
    end
end

@multitrace (macro with 2 methods)

In [14]:
@macroexpand @multitrace C G 4

quote
    #= In[13]:4 =#
    let var"#368#A" = C, var"#369#G" = G
        #= In[13]:5 =#
        begin
            var"#370###1181" = var"#369#G"[1]
            var"#371###1182" = var"#369#G"[2]
            var"#372###1183" = var"#369#G"[3]
            var"#373###1184" = var"#369#G"[4]
            var"#374###1185" = var"#368#A"
            var"#375###1189" = Main.eltype(var"#374###1185")
            var"#376###1193" = Main.promote_type(Main.promote_type(Main.eltype(var"#370###1181"), Main.eltype(var"#371###1182")), Main.eltype(var"#372###1183"), Main.typeof(true))
            var"#377###1197" = Main.promote_type(Main.eltype(var"#370###1181"), Main.eltype(var"#371###1182"), Main.typeof(true))
            var"#378###1194" = var"#370###1181"
            var"#379###1195" = var"#371###1182"
            var"#380###1196" = TensorOperations.cached_similar_from_indices(Symbol("##1196"), var"#377###1197", (1, 2), (2, 3), (1, 2, 3), (4,), var"#378###1194", var"#379###1195", :N, :N)
            va

In [15]:
C = similar(zeros(), j...)
@multitrace C G 4
A == B == C

true

* https://twitter.com/LirimyDh/status/1559364996865216512
* https://twitter.com/physics303/status/1559490168477732865

In [16]:
using StaticArrays

# ３次元配列を行列の配列に変換する
"Just specify that [an element of] the argument is an array of matrices"
struct MatrixArray end

"""
    R = matrixarray(S::AbstractArray{T,3})

`R[k][i, j] = S[i, k, j]`
"""
matrixarray(S) = [SMatrix{size(S[:, i, :])...}(S[:, i, :]) for i in axes(S, 2)]
#matrixarray(S) = [S[:, i, :] for i in axes(S, 2)] # without StaticArrays


# 行列積の最終段を省き、トレースを直接求める
reconst(G) = reconst(MatrixArray(), matrixarray.(G))
reconst(::MatrixArray, G) = _reconst(G...)

# Gs を真ん中で分けないと遅くなる
function _reconst(Gs...)
    h = length(Gs) ÷ 2
    _reconst(reduce(eachprod, Gs[begin:h]), reduce(eachprod, Gs[h+1:end]))
end

#_reconst(G1, G2, Gs...) = _reconst(eachprod(G1, G2), Gs...) # 遅い
_reconst(G1, G2) = trprod.(G1, expanddim(G2, G1))

"""
    trprod(A, B)

Returns `tr(A * B)`
"""
trprod(A, B) = dot(vec(A'), vec(B))

"""
    C = eachprod(A, B)

`C[i, j, k] = A[i, j] * B[k]`

`A, B, C :: Array{Matrix}`
"""
eachprod(A, B) = A .* expanddim(B, A)


"""
    Bx = expanddim(B, A)

`Bx = reshape(B, (1, 1, 1, m, n))` where `ndims(A) == 3`, `size(B) == (m, n)`
"""
expanddim(B, A) = reshape(B, (ntuple(_ -> 1, ndims(A))..., size(B)...))

expanddim

In [17]:
D = reconst(G)
A == B == C ≈ D

true

In [18]:
@macroexpand reconst4(G) = let X = G; @multitrace X 4 end

:(reconst4(G) = begin
          #= In[18]:1 =#
          let X = G
              #= In[18]:1 =#
              begin
                  #= In[10]:4 =#
                  let var"#674#G" = X
                      #= In[10]:5 =#
                      begin
                          var"#675###1233" = var"#674#G"[1]
                          var"#676###1234" = var"#674#G"[2]
                          var"#677###1235" = var"#674#G"[3]
                          var"#678###1236" = var"#674#G"[4]
                          var"#679###1241" = Main.promote_type(Main.promote_type(Main.promote_type(Main.eltype(var"#675###1233"), Main.eltype(var"#676###1234")), Main.eltype(var"#677###1235")), Main.eltype(var"#678###1236"), Main.typeof(true))
                          var"#680###1245" = Main.promote_type(Main.promote_type(Main.eltype(var"#675###1233"), Main.eltype(var"#676###1234")), Main.eltype(var"#677###1235"), Main.typeof(true))
                          var"#681###1249" = Main.promote_type(Main.el

In [19]:
using BenchmarkTools

reconst4(G) = @multitrace G 4

@show size.(G)
@show Threads.nthreads()

D = @btime reconst($G)
E = @btime reconst4($G)
F = @btime @tensor A[j1, j2, j3, j4] := $G[1][i1, j1, i2] * $G[2][i2, j2, i3] * $G[3][i3, j3, i4] * $G[4][i4, j4, i1]
A == B == C == E == F ≈ D

size.(G) = [(100, 4, 7), (7, 6, 8), (8, 5, 11), (11, 6, 100)]
Threads.nthreads() = 12
  663.500 μs (560 allocations: 693.36 KiB)
  235.400 μs (208 allocations: 24.72 KiB)
  236.100 μs (216 allocations: 25.19 KiB)


true

In [20]:
H = [G[mod1(k-1, 4)] for k in 1:4]

@show size.(H)
@show Threads.nthreads()

X = @btime reconst($H)
Y = @btime reconst4($H)
X ≈ Y

size.(H) = [(11, 6, 100), (100, 4, 7), (7, 6, 8), (8, 5, 11)]
Threads.nthreads() = 12
  169.000 μs (559 allocations: 388.44 KiB)
  48.700 μs (108 allocations: 13.03 KiB)


true